In [1]:
# Libraries needed for WebCam Face_Detection.
import cv2
import os
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import InceptionResnetV1,MTCNN
import sqlite3
from sklearn.neighbors import NearestNeighbors

In [2]:
os.getcwd()

'C:\\Users\\yeswa'

In [3]:
os.chdir("C:/Users/yeswa/OneDrive/Pictures/Face_Recognization_Photos")

In [4]:
#below variables are FaceNet related-> FaceNet creates Embedding Vectors of each face detected by MTCNN model.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')# This makes the enviroment use GPU if exist other wise CPU because of performance.
facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)# creating object for FaceNet Model.
mtcnn = MTCNN(keep_all=True,device=device)# Creating object for MTCNN model

In [5]:
folder_path = "C:/Users/yeswa/OneDrive/Pictures/Face_Recognization_Photos"
data = {}
def get_known_embedding(path):
    for folder_path in os.listdir(path):
        images_path = os.path.join(path,folder_path)
        if not os.path.isdir(images_path):
            continue
        embeddings = []
        for image in os.listdir(images_path):
            img_path = os.path.join(images_path,image)
            bgr_images = cv2.imread(img_path)
            bgr2rgb = cv2.cvtColor(bgr_images,cv2.COLOR_BGR2RGB)
            if bgr_images is None:
                continue
            detect_faces = mtcnn(bgr2rgb)
            if detect_faces is None:
                print("No Face is Detected:",{images_path})
                continue
            embedding = facenet(detect_faces).detach().cpu().squeeze(0)
            embeddings.append(embedding)
        if embeddings:
            data[folder_path] = embeddings
    return data

In [6]:
known_faces = get_known_embedding(folder_path)
known_faces

{'Seshu': [tensor([[-0.0049,  0.0041, -0.0049,  ..., -0.0503, -0.0565, -0.0272],
          [ 0.0239,  0.0163, -0.0447,  ..., -0.0121,  0.0059,  0.0178]]),
  tensor([[ 0.0029,  0.0060, -0.0029,  ..., -0.0474, -0.0413, -0.0146],
          [ 0.0494,  0.0216, -0.0568,  ..., -0.0174, -0.0063, -0.0278]]),
  tensor([[ 0.0282, -0.0175, -0.0147,  ..., -0.0015, -0.0170, -0.0701],
          [ 0.0094, -0.0220, -0.0457,  ..., -0.0638,  0.0108, -0.0202]]),
  tensor([[-0.0034, -0.0214, -0.0084,  ..., -0.0051, -0.0346, -0.0400],
          [-0.0114, -0.0322, -0.0535,  ..., -0.0546, -0.0107, -0.0266]]),
  tensor([[-0.0121, -0.0357, -0.0303,  ..., -0.0111, -0.0044, -0.0307],
          [ 0.0029, -0.0604, -0.0081,  ...,  0.0026,  0.0100, -0.0842]])],
 'Umar': [tensor([ 0.0098,  0.0355, -0.0478, -0.0101,  0.0008, -0.0642,  0.0712,  0.0200,
           0.0238, -0.0045,  0.0362, -0.0493,  0.0223, -0.0366,  0.0336, -0.0018,
          -0.0236,  0.0013, -0.0024,  0.0660,  0.1081, -0.0094, -0.0472,  0.0340,
      

In [17]:
all_embeddings = []
labels = []
for name,embedding in known_faces.items():
    for emb in embedding:
        if emb is None:
            continue
        emb_np = emb.detach().cpu().numpy().flatten()
        if emb_np.shape != (512,):
            continue
        all_embeddings.append(emb_np)
        labels.append(name)
all_embeddings = np.vstack(all_embeddings)
nn = NearestNeighbors(n_neighbors=1,metric='euclidean')

NearestNeighbors(metric='euclidean', n_neighbors=1)

In [21]:
labels = set(labels)
labels

{'Umar', 'Yeshwanth'}

In [23]:
nn.fit(all_embeddings,labels)

NearestNeighbors(metric='euclidean', n_neighbors=1)

In [27]:
# below code is read frames of webcam
url = "http://100.88.518.8080/video"
cap = cv2.VideoCapture(url) # initate the webcam
while True: # if flow is true then go inside the loop
        ret,frame = cap.read() # ret->True/Fasle of VideoCapturing if capturing True not capturing False
        if not ret: #condition on frames are getting or not.
            break # if condition true then break flow here itself.
        boxes,prob = mtcnn.detect(frame)# Detects faces from CC Camera and shows green box around face->box,prob->indicates accuracy of face >=95 then it is face.
        if boxes is not None:
            for box in boxes:
                x1,y1,x2,y2 = [int(coord) for coord in box]
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
                # Above loop and
                #two lines are going to display green line around the face lively.
                face = frame[y1:y2,x1:x2]# This variable consist of that face numerical vector
                face_rgb = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
                face_tensor = mtcnn(face_rgb)
                if face_tensor is None:
                    continue
                if len(face_tensor) == 3:
                        batch_tensor = face_tensor.unsqueeze(0).to(device)
                else:
                    batch_tensor = face_tensor.to(device)
                live_embedding = facenet(batch_tensor).detach().cpu().squeeze(0).reshape(1,-1)
                dist,idx = nn.kneighbors(live_embedding)
                if dist[0][0]<0.8:
                    print("Known Face")
                else:
                    print("Unknown")
        cv2.imshow("Frame",frame)# Display the image from vectors.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()